In [8]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from matplotlib import gridspec


df=input_data.read_data_sets('/tmp/data/',one_hot=True)

#parameter 
img_dim=784  #28*28=784
gen_dim=256
dis_dim=256
y_dim=10
noise_dim=100

gen_input=tf.placeholder(tf.float32,shape=[None,noise_dim])
dis_input=tf.placeholder(tf.float32,shape=[None,img_dim])
y_input=tf.placeholder(tf.float32,shape=[None,y_dim])

batch_size=128
num_step=40000
learning_rate=2e-4
display_step=2000

def weight_init(shape):
    return tf.random_normal(shape=shape , stddev=1./tf.sqrt(shape[0]/2.))


#weight and bias
W={"w1":tf.Variable(weight_init([noise_dim + y_dim,gen_dim])),
   "w2":tf.Variable(weight_init([gen_dim,img_dim])),
   "w3":tf.Variable(weight_init([img_dim +y_dim ,dis_dim])),
   "w4":tf.Variable(weight_init([dis_dim,1]))}

b={"b1":tf.Variable(tf.zeros([gen_dim])),
   "b2":tf.Variable(tf.zeros([img_dim])),
   "b3":tf.Variable(tf.zeros([dis_dim])),
   "b4":tf.Variable(tf.zeros([1]))}


#model architecture
def gen_fun(x,y):
    inputs=tf.concat(axis=1 , values=[x,y])
    li=tf.add(tf.matmul(inputs,W["w1"]),b['b1'])
    li=tf.nn.relu(li)
    li=tf.add(tf.matmul(li,W["w2"]),b['b2'])
    li=tf.nn.sigmoid(li)
    return li


def dis_fun(x,y):
    inputs=tf.concat(axis=1 , values=[x,y])
    l2=tf.add(tf.matmul(inputs,W["w3"]),b['b3'])
    l2=tf.nn.relu(l2)
    l2=tf.add(tf.matmul(l2,W["w4"]),b['b4'])
    l2=tf.nn.sigmoid(l2)
    return l2

#cost function optimization and model evaluation

gen_out=gen_fun(gen_input,y_input)
dis_real_out=dis_fun(dis_input,y_input)
dis_fake_out=dis_fun(gen_out,y_input)

cost_gen=-tf.reduce_mean(tf.log(dis_fake_out + .00001))
cost_dis=-tf.reduce_mean(tf.log(dis_real_out + .00001) + tf.log(1. - dis_fake_out + .00001))

var_gen =[W['w1'],W['w2'],b['b1'],b['b2']]
var_dis=[W['w3'],W['w4'],b['b3'],b['b4']]

training_gen=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost_gen,var_list=var_gen)

training_dis=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost_dis,var_list=var_dis)

#initialize the variable
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for step in range(1,num_step+1):
        
        batch_x,y_label=df.train.next_batch(batch_size)
        
        #generate the noise sample
        
        noise_temp=np.random.uniform(-1.,1.,size=[batch_size,noise_dim])
        
        feed_dict={dis_input:batch_x,y_input : y_label,gen_input:noise_temp}
        
        _,_,gl,dl=sess.run([training_gen,training_dis,cost_gen,cost_dis],feed_dict=feed_dict)
        
        if step % display_step==0 or step==1 :
            
            print('step-%i : generator_loss-%f : discriminator_loss-%f' % (step , gl,dl))
            
    print('optimization finished')

def generate_plot(samples):
    fig = plt.figure(figsize=(4,4))
    gs=gridspec.GridSpec(4,4)
    gs.update(wspace=.05 , hspace=.05)
    for i,sample in enumerate(samples):
        ax=plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticks([])
        ax.set_yticks([])
        #ax.set_aspects('equal')
        plt.imshow(sample.reshape(28,28), cmap='gray')
    return fig


def create(inp):
        feature_map={"t-shirt":0 , "trouser":1 , "pullover":2 , "dress":3 , "coat":4 , "sandal":5 , "shirt":6 , "sneaker":7 , "bag":8 , "ankel boot":9}
        samples=16
        z_noise=np.random.uniform(-1.,1.,size=[samples,noise_dim])
        y_labels=np.zeros(shape=[samples , y_dim])
        y_labels[:,feature_map[inp]]=1
        generate_samples =sess.run(gen_out , feed_dict={gen_input:z_noise,y_input:y_labels})
        generate_plot(generate_samples)

create("trouser")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
step-1 : generator_loss-0.970312 : discriminator_loss-1.246512
step-2000 : generator_loss-4.930574 : discriminator_loss-0.023635
step-4000 : generator_loss-5.139311 : discriminator_loss-0.079288
step-6000 : generator_loss-5.334298 : discriminator_loss-0.047676
step-8000 : generator_loss-4.383492 : discriminator_loss-0.094386
step-10000 : generator_loss-4.551865 : discriminator_loss-0.159159
step-12000 : generator_loss-4.837944 : discriminator_loss-0.105127
step-14000 : generator_loss-4.964076 : discriminator_loss-0.105384
step-16000 : generator_loss-5.579608 : discriminator_loss-0.074532
step-18000 : generator_loss-4.238797 : discriminator_loss-0.271322
step-20000 : generator_loss-5.229581 : discriminator_loss-0.123973
step-22000 : generator_loss-4.318390 : discriminator_loss-0.122044
step-24000 : g

RuntimeError: Attempted to use a closed Session.